In [1]:
from IPython.core.display import display, HTML
display(HTML('<h1>Neural Network</h1>'))


In [2]:
display(HTML('<h2>Imports</h2>'))

In [8]:
import pandas as pd
import numpy as np
import random as rnd



In [26]:
display(HTML('<h2>Load Training Data</h2>'))

In [14]:
train = pd.read_csv("clean_meta.csv")


In [27]:
display(HTML('<h2> Split Into Train And Test Set</h2>'))

In [16]:
from sklearn.model_selection import train_test_split
train = train.dropna()
train, test = train_test_split(train, test_size = 0.25)


In [18]:
X=train[['age','sex','s0','s1','s2','s3','s4','s5','s6']]
y=train['diagnosis']

In [28]:
display(HTML('<h2>Reshape Output</h2>'))

In [19]:
y = y.reshape(len(X), 1)

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [29]:
display(HTML('<h2>Layers</h2>'))

In [30]:
input_layer_size=X.shape[1]
hidden_layer_size=6
num_labels=7
counter=0


In [31]:
display(HTML('<h2>Sigmoid</h2>'))

In [ ]:
def sigmoid(z):  
    return 1 / (1 + np.exp(-z))


In [32]:
display(HTML('<h2>Sigmoid Gradient</h2>'))

In [ ]:
def sigmoidGradient(z):
    g=np.matrix(np.zeros(z.shape))
    g=np.multiply(sigmoid(z),(1-sigmoid(z)))
    return g
        

In [33]:
display(HTML('<h2>Forward Propogation</h2>'))

In [ ]:
def forwardpropagate(X,theta1,theta2):

   # theta1 = np.matrix(np.reshape(params[:hidden_layer_size * (input_layer_size + 1)], (hidden_layer_size, (input_layer_size + 1))))  
    #theta2 = np.matrix(np.reshape(params[hidden_layer_size * (input_layer_size + 1):], (num_labels, (hidden_layer_size + 1))))
    a1=np.matrix(np.insert(X,0,1,axis=1))
    #print("a1:\t"+str(a1.shape))
    z2=np.matmul(a1,theta1.T)
    a2=sigmoid(z2)
    a2=np.insert(a2,0,1,axis=1)
    #print("a2:\t"+str(a2.shape))
    z3=np.matmul(a2,theta2.T)
    a3=sigmoid(z3)
    return a3

In [34]:
display(HTML('<h2>Forward Propogation + Backward Propogation Model</h2>'))

In [21]:
def nnCostRegGrad(params,input_layer_size,hidden_layer_size,num_labels,X,y,lam):
    theta1 = np.matrix(np.reshape(params[:hidden_layer_size * (input_layer_size + 1)], (hidden_layer_size, (input_layer_size + 1))))  
    theta2 = np.matrix(np.reshape(params[hidden_layer_size * (input_layer_size + 1):], (num_labels, (hidden_layer_size + 1))))
    global counter
    #print(theta1.shape)
    #print(theta2.shape)
    m=X.shape[0]
    X=np.matrix(X)
    a1=np.matrix(np.insert(X,0,1,axis=1))
    #print("a1:\t"+str(a1.shape))
    z2=np.matmul(a1,theta1.T)
    a2=sigmoid(z2)
    a2=np.insert(a2,0,1,axis=1)
    #print("a2:\t"+str(a2.shape))
    z3=np.matmul(a2,theta2.T)
    a3=sigmoid(z3)
  
                       
    y2=np.copy(y)        
    
    #print("y2:\t"+str(y2.shape))
    y2=np.matrix(y2)  
    a3=np.matrix(a3)
    a2=np.matrix(a2)
    a1=np.matrix(a1)
    theta1=np.matrix(theta1)
    theta2=np.matrix(theta2)
    #print(y2)
    M=np.multiply(-y2.T,np.log(a3))-np.multiply((1-y2.T),np.log(1-a3))
    l=np.sum(np.sum(M))
    J=l/m
    
    t1=np.power(theta1,2)
    t2=np.power(theta2,2)
    reg1=np.sum(np.sum(t1[:,2:theta1.shape[1]]))
    reg2=np.sum(np.sum(t2[:,2:theta1.shape[1]]))
    
    reg=reg1+reg2
    
    finreg=(lam*reg)/(2*m)
    J+=finreg
    
    
    
    #print(J)
    yy=np.matrix(y2.T)
    b=np.matrix(np.insert(X,0,1,axis=1))
    findel1=np.matrix(np.zeros(theta1.shape))
    findel2=np.matrix(np.zeros(theta2.shape))
    #print(findel1.shape)
    #print(findel2.shape)
    percentage=100*(counter/250)
    print(percentage)
    counter=counter+1
    for t in range(X.shape[0]):
        aa1=np.matrix(b[t])
        
        zz2=np.matmul(aa1,theta1.T)
        aa2=sigmoid(zz2)
        aa2=np.insert(aa2,0,1,axis=1)
        
        #print("aa2:\t"+str(aa2.shape))
        zz3=np.matmul(aa2,theta2.T)
        aa3=sigmoid(zz3)
        #print(aa3)
        #print(yy[t].shape)
        d3=aa3-yy[t]
        #print("d3:\t"+str(d3.shape))
        d2=np.multiply(np.matmul(d3,theta2),sigmoidGradient(np.insert(zz2,0,1)))
        #print("d2:\t"+str(d2.shape))
        #print(findel1.shape)
        #print("d2 before:\t"+str(d2.shape))
        d2=d2[:,1:]
     #   print("d2:\t"+str(d2.shape))
        findel1=findel1+np.matmul(d2.T,aa1)
        #print(findel1)
        findel2=findel2+np.matmul(d3.T,aa2)
        #print(findel2)
    Theta1_grad=np.matrix(np.zeros(theta1.shape))
    Theta2_grad=np.matrix(np.zeros(theta2.shape))
    Theta1_grad=findel1/m
    Theta2_grad=findel2/m
    
    #print(Theta1_grad.shape,Theta2_grad.shape)
    
    Theta1_grad[:,1:Theta1_grad.shape[1]]=Theta1_grad[:,1:]+((lam*theta1[:,1:]))/m
    Theta2_grad[:,1:Theta2_grad.shape[1]]=Theta2_grad[:,1:]+((lam*theta2[:,1:]))/m
    
    grad = np.concatenate((np.ravel(Theta1_grad), np.ravel(Theta2_grad)))
    
    return J,grad




In [35]:
display(HTML('<h2>Convert Y to one-hot representation</h2>'))

In [22]:
y2=np.zeros((num_labels,X.shape[0]))
   
for i in range(0,X.shape[0]):
    for j in range(0,num_labels):
        if(y[i]==j):
            y2[j][i]=1

params = (np.random.random(size=hidden_layer_size * (input_layer_size + 1) + num_labels * (hidden_layer_size + 1)) - 0.5) * 0.25

# unravel the parameter array into parameter matrices for each layer
J,grad=nnCostRegGrad(params,input_layer_size,hidden_layer_size,num_labels,X,y2,lam=1)
y





0.0


array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [36]:
display(HTML('<h2>Run Gradient Descent</h2>'))

In [23]:
from scipy.optimize import minimize
lam=1
# minimize the objective function
fmin = minimize(fun=nnCostRegGrad, x0=params, args=(input_layer_size, hidden_layer_size, num_labels, X, y2,lam),  
                method='TNC', jac=True, options={'maxiter': 250})
fmin


# In[50]:

X = np.matrix(X)  
theta1 = np.matrix(np.reshape(fmin.x[:hidden_layer_size * (input_layer_size + 1)], (hidden_layer_size, (input_layer_size + 1))))  
theta2 = np.matrix(np.reshape(fmin.x[hidden_layer_size * (input_layer_size + 1):], (num_labels, (hidden_layer_size + 1))))

h = forwardpropagate(X,theta1,theta2)  
y_pred = np.array(np.argmax(h, axis=1))  
y,y_pred


# In[51]:

correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]  
accuracy = (sum(map(int, correct)) / float(len(correct)))  
print ('accuracy = {0}%'.format(accuracy * 100))

0.4
0.8
1.2
1.6
2.0
2.4
2.8000000000000003
3.2
3.5999999999999996
4.0
4.3999999999999995
4.8
5.2
5.6000000000000005
6.0
6.4
6.800000000000001
7.199999999999999
7.6
8.0
8.4
8.799999999999999
9.2
9.6
10.0
10.4
10.8
11.200000000000001
11.600000000000001
12.0
12.4
12.8
13.200000000000001
13.600000000000001
14.000000000000002
14.399999999999999
14.799999999999999
15.2
15.6
16.0
16.400000000000002
16.8
17.2
17.599999999999998
18.0
18.4
18.8
19.2
19.6
20.0
20.4
20.8
21.2
21.6
22.0
22.400000000000002
22.8
23.200000000000003
23.599999999999998
24.0
24.4
24.8
25.2
25.6
26.0
26.400000000000002
26.8
27.200000000000003
27.6
28.000000000000004
28.4
28.799999999999997
29.2
29.599999999999998
30.0
30.4
30.8
31.2
31.6
32.0
32.4
32.800000000000004
33.2
33.6
34.0
34.4
34.8
35.199999999999996
35.6
36.0
36.4
36.8
37.2
37.6
38.0
38.4
38.800000000000004
39.2
39.6
40.0
40.400000000000006
40.8
41.199999999999996
41.6
42.0
42.4
42.8
43.2
43.6
44.0
44.4
44.800000000000004
45.2
45.6
46.0
46.400000000000006
46.800

In [41]:
display(HTML('<h2>Testing</h2>'))

In [38]:
test=test.dropna()
fnx2=np.array(test[['age','sex','s0','s1','s2','s3','s4','s5','s6']])
testy=test['diagnosis']


In [27]:
h2 = forwardpropagate(fnx2,theta1,theta2)  
y_pred2 = np.array(np.argmax(h2, axis=1))  


In [39]:
display(HTML('<h2>F-SCORE</h2>'))

In [33]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_pred2,testy,average='weighted')

/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(0.9575229281989843, 0.9201060204468005, 0.9313156293336067, None)